In [12]:
from torchvision import datasets, transforms
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import torch
from tqdm import tqdm
import time
import random

In [13]:
def dist_man(x, y):
    #return torch.sum((x-y)**2)**0.5
    #return torch.sum((x-y)**4)**0.5
    #return torch.sum((x-y)**6)**0.5 #best
    #return 1-torch.cosine_similarity(x,y,dim=0) #95.5 k=11
    # return torch.dist(x, y, 6)#best k=6
    return torch.dist(x, y, 1)
    

In [14]:
def dist_euc(x,y):
    return torch.dist(x, y, 2)

In [15]:
def dist_cos(x,y):
    return 1-torch.cosine_similarity(x,y,dim=0)

In [16]:
def dist_L3(x,y):
    return torch.dist(x, y, 3)

In [17]:

# iteration func
def KNN_by_iter(train_x, train_y, test_x, test_y, k,get_dist):
    pca=PCA(0.9)
    pca.fit(train_x)
    train_x_reduct=torch.from_numpy(pca.transform(train_x))
    test_x_reduct=torch.from_numpy(pca.transform(test_x))
    since = time.time()
    # cal distance
    res = []
    for x in tqdm(test_x_reduct):
        dists = []
        for y in train_x_reduct:
            dists.append(get_dist(x,y).view(1))
        
        idxs = torch.cat(dists).argsort()[:k]
        res.append(np.bincount(np.array([train_y[idx] for idx in idxs])).argmax())

    print(res[:10])#?
    print("acc",accuracy_score(test_y, res))

    time_elapsed = time.time() - since
    print('KNN iter training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
        

In [21]:
def Run_Knn(train_dataset,dist_para,test_x,test_y):
    train_x = []
    train_y = []
    
    # for i in range(len(train_dataset)):
    index = [i for i in range(len(test_dataset))] 
    random.shuffle(index)
    for i in index[0:10000]:
        img, target = train_dataset[i]
        img=img.squeeze()
        # plt.imshow(img,'gray')
        # plt.title(target)
        # plt.show()
        # break
        
        train_x.append(img.view(-1))
        train_y.append(target)
        # if i > 10000:
        #     break
    get_dist=dist_para
    KNN_by_iter(torch.stack(train_x), train_y, torch.stack(test_x), test_y, 4,get_dist)

In [22]:
if __name__ == "__main__":

    train_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=True,download=True)
    test_dataset = datasets.MNIST(root="./data", transform=transforms.ToTensor(), train=False,download=True)

    test_x = [] 
    test_y = []
    for i in range(len(test_dataset)):
        img, target = test_dataset[i]
        img=img.squeeze()      
        test_x.append(img.view(-1))
        test_y.append(target)

    dist_lst=[dist_man,dist_euc,dist_cos,dist_L3]
    for dist_para in dist_lst:
        Run_Knn(train_dataset,dist_para,test_x,test_y)
    
    


torch.Size([10000, 85])
torch.Size([10000, 85])
torch.Size([10000, 85])
torch.Size([10000, 85])
